# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Ames Housing dataset.

## Objectives

You will be able to:
- Implement interaction terms in Python using the `sklearn` and `statsmodels` packages 
- Interpret interaction variables in the context of a real-world problem 

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below: 

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [85]:
regression = LinearRegression()

Create a baseline model which includes all the variables we selected from the Ames housing data set to predict the house prices. Then use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [86]:
ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

In [87]:
# continuous variables
ames_cont = ames[continuous]
ames_log = np.log(ames_cont)
log_names = [f'{feature}_log' for feature in continuous]
ames_log.columns = log_names
ames_log.head()

,LotArea_log,1stFlrSF_log,GrLivArea_log,SalePrice_log
0,9.041922,6.752270,7.444249,12.247694
1,9.169518,7.140453,7.140453,12.109011
2,9.328123,6.824374,7.487734,12.317167
3,9.164296,6.867974,7.448334,11.849398
4,9.565214,7.043160,7.695303,12.429216


In [88]:
# helper function to normalize
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

In [89]:
# normalize ames_log
ames_log_norm = ames_log.apply(normalize)

In [90]:
# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

In [91]:
# concat continuous with categorical
preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)
X = preprocessed.drop(columns=['SalePrice_log'])
y = preprocessed['SalePrice_log']

In [92]:
#get baseline model
regression = LinearRegression()
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X, y, cv=crossvalidation))
baseline

0.8174636844561984

## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the $R^2$. We'll look at the 3 interactions which have the biggest effect on our $R^2$, so print out the top 3 combinations.

You will create a `for` loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

Since there are so many different neighbourhoods we will exclude

In [93]:
from itertools import combinations

In [105]:
features = X.columns
features

Index(['LotArea_log', '1stFlrSF_log', 'GrLivArea_log', 'BldgType_2fmCon',
       'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE',
       'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA', 'SaleType_CWD',
       'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw',
       'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'MSZoning_FV',
       'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave',
       'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor',
       'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes',
       'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge',
       'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU',
       'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst',
       'Neighborhood_StoneBr

In [106]:
combos = combinations(features, 2)
all_interactions = [combo for combo in combos]

In [111]:
# store interaction results in dictionary
interactions = []

for i, interaction in enumerate(all_interactions):
    X_interact = X.copy()
    X_interact[interaction] = X[interaction[0]] * X[interaction[1]]
    r_squared = np.mean(cross_val_score(regression, X_interact, y, scoring='r2', cv=crossvalidation))
    
    if (r_squared > baseline):
        results[interaction] = r_squared
        print(interaction, r_squared)

('LotArea_log', '1stFlrSF_log') 0.8177808253078604
('LotArea_log', 'BldgType_2fmCon') 0.8181603441154108
('LotArea_log', 'KitchenQual_Fa') 0.8179028633875973
('LotArea_log', 'KitchenQual_Gd') 0.8185111933490097
('LotArea_log', 'KitchenQual_TA') 0.8181256011196583
('LotArea_log', 'SaleType_Con') 0.8175774314593662
('LotArea_log', 'SaleType_New') 0.8183259638768854
('LotArea_log', 'SaleType_Oth') 0.8187451822913502
('LotArea_log', 'SaleType_WD') 0.8184644282916084
('LotArea_log', 'MSZoning_RH') 0.817628286442118
('LotArea_log', 'MSZoning_RM') 0.817558725021885
('LotArea_log', 'Neighborhood_BrkSide') 0.8195228553736742
('LotArea_log', 'Neighborhood_Edwards') 0.8188308850559454
('LotArea_log', 'Neighborhood_Mitchel') 0.8196447680686683
('LotArea_log', 'Neighborhood_NoRidge') 0.8179939655000826
('LotArea_log', 'Neighborhood_NridgHt') 0.8192448579373913
('LotArea_log', 'Neighborhood_OldTown') 0.8179331439487916
('LotArea_log', 'Neighborhood_Sawyer') 0.8177448506410391
('LotArea_log', 'Neighb

('MSZoning_RH', 'Neighborhood_NAmes') 0.8185889697272388
('MSZoning_RH', 'Neighborhood_SWISU') 0.8183348892800494
('MSZoning_RH', 'Neighborhood_SawyerW') 0.8188104075307606
('MSZoning_RL', 'Neighborhood_BrkSide') 0.8180970632936317
('MSZoning_RL', 'Neighborhood_Crawfor') 0.8177183784826901
('MSZoning_RL', 'Neighborhood_Mitchel') 0.8181487352855108
('MSZoning_RL', 'Neighborhood_NAmes') 0.8185889697272388
('MSZoning_RL', 'Neighborhood_NridgHt') 0.8176030368352845
('MSZoning_RL', 'Neighborhood_SWISU') 0.8183348892800494
('MSZoning_RL', 'Neighborhood_Sawyer') 0.818182279723719
('MSZoning_RL', 'Neighborhood_SawyerW') 0.8188104075307606
('MSZoning_RM', 'Neighborhood_BrkSide') 0.8180970632936315
('MSZoning_RM', 'Neighborhood_Crawfor') 0.8187398397493473
('MSZoning_RM', 'Neighborhood_Edwards') 0.8176152721565246
('MSZoning_RM', 'Neighborhood_IDOTRR') 0.8180902031693108
('MSZoning_RM', 'Neighborhood_Mitchel') 0.8181487352855108
('MSZoning_RM', 'Neighborhood_NridgHt') 0.817498100206992
('MSZonin

In [118]:
sorted_dict = {k: v for k, v in sorted(results.items(), key=lambda item: item[1], reverse=True)}

In [119]:
sorted_dict

{('BldgType_Duplex', 'Street_Pave'): 0.8196563892558343,
 ('LotArea_log', 'Neighborhood_Mitchel'): 0.8196447680686683,
 ('GrLivArea_log', 'BldgType_Duplex'): 0.819620127969196,
 ('GrLivArea_log', 'KitchenQual_TA'): 0.8196192484429915,
 ('LotArea_log', 'Neighborhood_BrkSide'): 0.8195228553736742,
 ('GrLivArea_log', 'KitchenQual_Gd'): 0.8193241877118306,
 ('BldgType_Duplex', 'Neighborhood_Edwards'): 0.8192640599816852,
 ('LotArea_log', 'Neighborhood_NridgHt'): 0.8192448579373913,
 ('1stFlrSF_log', 'BldgType_Duplex'): 0.8191446667521737,
 ('LotArea_log', 'Neighborhood_Edwards'): 0.8188308850559454,
 ('MSZoning_RH', 'Neighborhood_SawyerW'): 0.8188104075307606,
 ('MSZoning_RL', 'Neighborhood_SawyerW'): 0.8188104075307606,
 ('1stFlrSF_log', 'Neighborhood_NridgHt'): 0.8187497960368522,
 ('LotArea_log', 'SaleType_Oth'): 0.8187451822913502,
 ('BldgType_TwnhsE', 'Neighborhood_StoneBr'): 0.8187427548029594,
 ('BldgType_TwnhsE', 'Neighborhood_Crawfor'): 0.8187398397493473,
 ('MSZoning_RM', 'Neighb

It looks like the top interactions involve the Neighborhood_Edwards feature so lets add the interaction between LotArea and Edwards to our model.

We can interpret this feature as the relationship between LotArea and SalePrice when the house is in Edwards or not.

## Visualize the Interaction

Separate all houses that are located in Edwards and those that are not. Run a linear regression on each population against `SalePrice`. Visualize the regression line and data points with price on the y axis and LotArea on the x axis.

In [ ]:
# Visualization code here

## Build a final model with interactions

Use 10-fold cross-validation to build a model using the above interaction. 

In [ ]:
# code here

Our $R^2$ has increased considerably! Let's have a look in `statsmodels` to see if this interactions are significant.

In [ ]:
# code here

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You should now understand how to include interaction effects in your model! As you can see, interactions can have a strong impact on linear regression models, and they should always be considered when you are constructing your models.